In [2]:
%matplotlib inline
import scipy.stats as st
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.pyplot import figure
import matplotlib as mpl
mpl.rcParams['figure.dpi'] = 400

# Проверка гипотезы распределения используя критерий Пирсона
. Сгенерировать выборку объёмом 100 элементов для нормального распределения $N(x, 0, 1)$. По сгенерированной выборке оценить параметры 𝜇 и 𝜎 нормального закона методом максимального правдоподобия.
В качестве основной гипотезы 𝐻0 будем считать, что сгенерированное
распределение имеет вид $N(x, \hat{\mu}, \hat{\sigma})$. Проверить основную гипотезу, используя критерий согласия $\chi^2 $
. В качестве уровня значимости взять
𝛼 = 0.05. Привести таблицу вычислений $\chi^2 $
.
Исследовать точность (чувствительность) критерия $\chi^2 $
- сгенерировать выборки равномерного распределения и распределения Лапласа
малого объема (например, 20 элементов). Проверить их на нормальность.


In [3]:
# размер выборки
N = 20

# выборка:
data = st.norm.rvs(size=N)



Метод максимального правдоподобия (о.м.п.)

In [4]:
def OMP(array):
    mean = array.mean()
    std = array.std()
    return {
        'mean' : mean,
        'std' : std
    }

Критерий Пирсона для оценки гипотезы о распределении

In [5]:
def Pirson(data,func):
    
    N = len(data)
    
    # extimation of sigma and mean
    L = OMP(data)
    
    try:
        func = func(L['mean'],L['std'])
    except TypeError:
        print("cont")
    
    print("mean : ",L['mean'])
    print("std : ",L['std'])
    
    


    # степени свободы
    CONST_DEGREES_OF_FREEDOM = 6

    # начальное значение и конечное значение отрезка
    start = min(data)
    end = max(data)
    
    start = -2.45
    end = 2.14
    val = np.linspace(start,end,CONST_DEGREES_OF_FREEDOM - 1)
    
    
    probablity = []
    probablity_sel = []
    for i in range(0,len(val) + 1):
        if i == 0:
            probablity.append(func.cdf(val[i]))
            probablity_sel.append(sum(data <= val[i]))
        elif i == len(val):
            probablity.append(1 - func.cdf(val[i - 1]))
            probablity_sel.append(sum(data > val[i - 1]))
        else:
            probablity.append(func.cdf(val[i]) -func.cdf(val[i - 1]) )
            probablity_sel.append(sum(np.bitwise_and(data <= val[i],data > val[i - 1])))
    
    ni__npi = 0
    npi = 0
    pi = 0
    hi = 0
    n = 0
    print("n\tpi\tnpi\tni - npi\t(ni - npi)/(npi)")
    for i in range(CONST_DEGREES_OF_FREEDOM):
        hi += (probablity_sel[i] - N*probablity[i])**2/(N * probablity[i] + 0.001)
        n += probablity_sel[i]
        pi += probablity[i]
        npi += N*probablity[i]
        ni__npi += probablity_sel[i] - N*probablity[i]
        print(str(probablity_sel[i]),
            format(probablity[i],'.2f'),
            format(N*probablity[i],'.2f'),
            format(probablity_sel[i] - N*probablity[i],'.2f') + "\t",
            format((probablity_sel[i] - N*probablity[i])**2/(N * probablity[i]  +0.000001),'.2f') ,
             sep="\t")
    print(str(n),
        format(pi,'.2f'),
        format(npi,'.2f'),
        format(ni__npi,'.2f') + "\t",
        format(hi,'.2f') ,
         sep="\t")
    print(val)
    return hi,CONST_DEGREES_OF_FREEDOM



In [6]:
Pirson(data,st.norm)

mean :  -0.24112777950542644
std :  1.1656305199969916
n	pi	npi	ni - npi	(ni - npi)/(npi)
1	0.03	0.58	0.42		0.30
1	0.15	3.04	-2.04		1.37
10	0.35	6.96	3.04		1.32
4	0.33	6.51	-2.51		0.97
4	0.12	2.49	1.51		0.92
0	0.02	0.41	-0.41		0.41
20	1.00	20.00	0.00		5.29
[-2.45   -1.3025 -0.155   0.9925  2.14  ]


(5.294484287633448, 6)

In [7]:
Pirson(data,st.laplace)

mean :  -0.24112777950542644
std :  1.1656305199969916
n	pi	npi	ni - npi	(ni - npi)/(npi)
1	0.08	1.50	-0.50		0.17
1	0.13	2.52	-1.52		0.92
10	0.33	6.69	3.31		1.64
4	0.29	5.82	-1.82		0.57
4	0.11	2.17	1.83		1.53
0	0.06	1.30	-1.30		1.30
20	1.00	20.00	0.00		6.12
[-2.45   -1.3025 -0.155   0.9925  2.14  ]


(6.120177990500319, 6)

In [8]:
alpha = -3**0.5
scale = -alpha * 2
Pirson(data,st.uniform(alpha,scale))

cont
mean :  -0.24112777950542644
std :  1.1656305199969916
n	pi	npi	ni - npi	(ni - npi)/(npi)
1	0.00	0.00	1.00		1000000.00
1	0.12	2.48	-1.48		0.88
10	0.33	6.63	3.37		1.72
4	0.33	6.63	-2.63		1.04
4	0.21	4.27	-0.27		0.02
0	0.00	0.00	0.00		0.00
20	1.00	20.00	-0.00		1003.66
[-2.45   -1.3025 -0.155   0.9925  2.14  ]


(1003.6588813005613, 6)